In [1]:
import dash
from dash import dcc, html, Input, Output, State
import plotly.express as px
import pandas as pd
from datetime import datetime

In [2]:
actual = pd.read_csv("actual_data.csv")
forecast = pd.read_csv("forecast_data.csv")

# Define headers
headers = ["Date", "average_sentiment"]

# Assign headers to df1
forecast.columns = headers

# Convert the 'Date' column to datetime
actual['Date'] = pd.to_datetime(actual['Date'])
forecast['Date'] = pd.to_datetime(forecast['Date'])

In [3]:
# Function to create the plot
def create_dashboard(df1, df2, aggregation='W'):
    # Aggregate data based on the selected aggregation
    df1_agg = aggregate_data(df1, aggregation)
    df2_agg = aggregate_data(df2, aggregation)

    # Create subplots
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add traces
    fig.add_trace(go.Scatter(x=df1_agg.index, y=df1_agg['average_sentiment'], name='Real Info', line=dict(color='blue')))
    fig.add_trace(go.Scatter(x=df2_agg.index, y=df2_agg['average_sentiment'], name='Forecasted Info', line=dict(color='red')))

    # Add layout
    fig.update_layout(title_text="Data Visualization Dashboard", xaxis_title="Date", yaxis_title="Average Score")
    return fig

In [4]:
# Function to aggregate data
def aggregate_data(df, freq):
    return df.resample(freq, on='Date').mean()

In [5]:
app = dash.Dash(__name__)

In [6]:
app.layout = html.Div([
    dcc.Dropdown(
        id='frequency-dropdown',
        options=[
            {'label': 'Weekly', 'value': 'W'},
            {'label': 'Monthly', 'value': 'M'},
            {'label': 'Quarterly', 'value': 'Q'}
        ],
        value='W',  # default value
    ),
    dcc.Dropdown(
        id='dataframe-dropdown',
        options=[
            {'label': 'Forecast', 'value': 'forecast'},
            {'label': 'Actual', 'value': 'actual'},
            {'label': 'Both', 'value': 'both'}
        ],
        value='both'  # default value
    ),
    dcc.DatePickerRange(
        id='date-picker-range',
        start_date=datetime(2009, 4, 12),  # Set the default start date
        end_date=datetime(2009, 9, 20),  # Set the default end date
        min_date_allowed=datetime(2009, 4, 12),  # Set the earliest selectable date
        max_date_allowed=datetime(2009, 9, 20),  # Set the latest selectable date
        initial_visible_month=datetime(2009, 4, 12),  # Set the initial month displayed
        end_date_placeholder_text='End Period',
        calendar_orientation='horizontal',
    ),
    dcc.Graph(id='timeseries-graph')
])

In [7]:
@app.callback(
    Output('timeseries-graph', 'figure'),
    [Input('frequency-dropdown', 'value'),
     Input('dataframe-dropdown', 'value'),
     Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date')]
)
def update_graph(frequency, dataframe, start_date, end_date):
    # Based on the inputs, filter and/or aggregate the dataframes, and generate the plot
    # Here you would include your logic to filter the df_forecast and df_actual based on the selected dates
    # and frequency, then plot them using Plotly Express or Graph Objects
    # Aggregate data based on the selected aggregation
    filtered_df2 = forecast[(forecast['Date'] >= start_date) & (forecast['Date'] <= end_date)]
    filtered_df1 = actual[(actual['Date'] >= start_date) & (actual['Date'] <= end_date)]
    
    
    
    # Reset the index to turn the 'Date' index into a column
    
    

    # Sample plot generation (replace this with your actual plotting logic)
    fig = px.line()  # or use go.Figure() for more complex plots

    if dataframe in ['forecast', 'both']:
        if not filtered_df2.empty:
            df2 = aggregate_data(filtered_df2, frequency)
            df2 = df2.reset_index()
        # Filter and resample df_forecast according to frequency and date range
        # ...
            fig.add_scatter(x=df2['Date'], y=df2['average_sentiment'], name='Forecast')

    if dataframe in ['actual', 'both']:
        if not filtered_df1.empty:
            df1 = aggregate_data(filtered_df1, frequency)
            df1 = df1.reset_index()
        # Filter and resample df_actual according to frequency and date range
        # ...
            fig.add_scatter(x=df1['Date'], y=df1['average_sentiment'], name='Actual')
            
    fig.update_layout(title='Average Sentiment Over Time', xaxis_title='Date', yaxis_title='Average Sentiment')

    return fig

http://127.0.0.1:8050/

In [8]:
if __name__ == '__main__':
    app.run_server(debug=True)